# Natural Gas Data Collection
### What we are building
We will build a Web Crawler to collect data for us that we will then build scripts to clean and analyze the data we collect. The data we are collecting in this example is from https://www.eia.gov a U.S Government agency, Energy Information Administration. They have a vast supply of data for all sorts of enrgies like Crude Oil, Natural Gas, Coal, Heating Oil, etc, but for this example we will only be using Natural Gas data.
### Modules
 - bs4    (https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [2]:
# Importing Modules
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from datetime import datetime
import pandas as pd
import csv

### Configuring Soup
We set the url equal to the website with the target data we are trying to collect. Next we define the Client to request are target url, we then create the soup to read all the html in the url, And lastly closing the client.

In [3]:
# Config for bs4
url =  'https://www.eia.gov/dnav/ng/ng_sum_lsum_dcu_nus_m.htm' # Target Website
uClient = uReq(url) # Defining 
page_soup = soup(uClient.read(), "html.parser") # Creating the soup
uClient.close() #Closing 

### Collecting Raw Data
We now define the data we are looking to collect, if you try to reproduce this project with different data you can find the data be inspecting the elements in your browser. The target data in this example is in the following html tags.
- < td class="Current2" > TARGET_DATA < /td > 

To find all the data in these html tags we use the following
- find.All('td', {'class' : 'Current2'})

Now we have our target data, but it's raw data we need to clean it for efficient future use.

In [4]:
# Finding Target Data
Target_Data = page_soup.findAll('td', {'class' : 'Current2'})
sample = Target_Data[:5]
print('Sample of Data Collected!!! ', sample)

Sample of Data Collected!!!  [<td class="Current2" width="76">NA</td>, <td class="Current2" width="76">1.50</td>, <td class="Current2" width="76">1.48</td>, <td class="Current2" width="76">1.48</td>, <td class="Current2" width="76">3.17</td>]


### Defining and Converting the html data to plain text data
Now we want to define our data and filter out the html tags from the data, so we just have the values and annoying < td > etc. we can only use the built in bs4 function .text with single elements, CANT use with a list.
- This works!
 - test = test_data[1].text


- This does not work!
 - test = test_data[1:10].text

In [5]:
# Defining and Converting html data to plain text
## Prices
prices_i = Target_Data[1].text
prices_i_bp = Target_Data[2].text
prices_i_lng = Target_Data[3].text
prices_e = Target_Data[4].text
prices_e_bp = Target_Data[5].text
prices_e_lng = Target_Data[6].text
prices_citygate = Target_Data[7].text
prices_resid = Target_Data[8].text
prices_commer = Target_Data[9].text
prices_indust = Target_Data[10].text
prices_elec_power = Target_Data[11].text
## Production
prod_gw = Target_Data[12].text
prod_mp = Target_Data[20].text
prod_ngpl = Target_Data[21].text
prod_dry = Target_Data[22].text
## Import/Export Volume (McF)
vol_i_imports = Target_Data[23].text
vol_i_bp = Target_Data[24].text
vol_i_lng = Target_Data[25].text
vol_e_exports = Target_Data[26].text
vol_e_bp = Target_Data[27].text
vol_e_lng = Target_Data[28].text
## Storage Volume (McF)
stor_tc = Target_Data[29].text
stor_gs = Target_Data[30].text
stor_bg = Target_Data[31].text
stor_wg = Target_Data[32].text
stor_Injec = Target_Data[33].text
stor_Withd = Target_Data[34].text
stor_nw = Target_Data[35].text 
## Consumption Volume (McF)
consump_tc = Target_Data[36].text
consump_lpf = Target_Data[37].text
consump_pdu = Target_Data[38].text
consump_d2c = Target_Data[39].text
consump_resid = Target_Data[40].text
consump_commer = Target_Data[41].text
consump_indust = Target_Data[42].text
consump_vf = Target_Data[43].text
consump_ep = Target_Data[44].text

### Creating a series of if statements to figure out the current month
A big problem we run into with this data is it's lagging 2 months behind each release date, so to acurratley date the new data our bot collects we have to create a chain of if statements to subtract the current month by 2. For example, the most recent (april) data was published June 30th. So the next data release on July 31st will be for Mays data from 2 months ago.
- x_month = the current month (1-12)
- formula = x_month - 2 months = the month of the data
- if x_month = 12 
    - then formula = 10 = October 

In [10]:
# Defining x Date
x_month = datetime.now().month
formula = x_month - 2

# If statements to tell the month
if formula == 1:
    date_ = '2020-01-1'
if formula == 2:
    date_ = '2020-02-1'
if formula == 3:
    date_ = '2020-03-1'
if formula == 4:
    date_ = '2020-04-1'
if formula == 5:
    date_ = '2020-05-1'
if formula == 6:
    date_ = '2020-06-1'
if formula == 7:
    date_ = '2020-07-1'
if formula == 8:
    date_ = '2020-08-1'
if formula == 9:
    date_ = '2020-09-1'
if formula == 10:
    date_ = '2020-10-1'
if formula == 11:
    date_ = '2020-11-1'
if formula == 12:
    date_ = '2020-12-1'
print(date_)

2020-05-1


### Creating Pandas DataFrame

In [13]:
# Defining Index
idex = (date_,)
# Defining Columns
col = ('Import Prices', 'Import Prices By Pipeline', 'Import Prices As Liquified NatGas', 'Export Prices', 'Export Prices By Pipeline', 'Export Prices As Liquified NatGas',
     'Citygate Prices', 'Residential Prices', 'Commercial Prices', 'Industrial Prices', 'Electric Power Prices', 'Gross Withdrawals', 'Marketed Production', 'NGPL Production', 
     'Dry Production', 'Import Volume(Mcf)', 'Import Volume By Pipeline', 'Import Volume As LNG', 'Export Volume(Mcf)', 'Export Volume By Pipeline', 'Export Volume As LNG', 
     'Total Capacity', 'Gas in Storage', 'Base Gas', 'Working Gas', 'Injections', 'Withdrawals', 'Net Withdrawals','Total Consumption', 'Lean & Plant Fuel', 'Pipeline & Distribution Use',
     'Delivered to Consumers', 'Residential', 'Commercial', 'Industrial', 'Vehicle Fuel', 'Electric Power')
# Defining Dataset
data = (prices_i, prices_i_bp, prices_i_lng, prices_e, prices_e_bp, prices_e_lng,
       prices_citygate, prices_resid, prices_commer, prices_indust, prices_elec_power, 
       prod_gw, prod_mp, prod_ngpl, prod_dry, vol_i_imports, vol_i_bp, vol_i_lng, vol_e_exports,
       vol_e_bp, vol_e_lng, stor_tc, stor_gs, stor_bg, stor_wg, stor_Injec, stor_Withd,
       stor_nw, consump_tc, consump_lpf, consump_pdu, consump_d2c, consump_resid, consump_commer,
       consump_indust, consump_vf, consump_ep)
# Creating DataFrame
df = pd.DataFrame([data], index=idex, columns=(col))
df

,Import Prices,Import Prices By Pipeline,Import Prices As Liquified NatGas,Export Prices,Export Prices By Pipeline,Export Prices As Liquified NatGas,Citygate Prices,Residential Prices,Commercial Prices,Industrial Prices,...,Net Withdrawals,Total Consumption,Lean & Plant Fuel,Pipeline & Distribution Use,Delivered to Consumers,Residential,Commercial,Industrial,Vehicle Fuel,Electric Power
2020-05-1,1.50,1.48,1.48,3.17,1.68,4.67,3.06,10.68,7.38,2.97,...,"-305,362","2,247,239","153,212","64,476","2,029,552","377,575","239,988","643,237","4,835","763,917"


In [17]:
# Exporting Dataset
df.to_csv('Natural Gas Data.csv', mode='a')